# Fetch and parse ACS benchmark results under a given directory
Each ACS benchmark run outputs a json file. This script collects all such files under a given root directory, parses them, and aggregates them into a more easily digestable pandas DataFrame.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

Set the local path to the root results directory:

In [2]:
RESULTS_ROOT_DIR = Path("/fast/groups/sf") / "folktexts-results" / "2024-07-03"

Set the local path to the root data directory (needed only to train baseline ML methods):

In [3]:
DATA_DIR = Path("/fast/groups/sf") / "data"

Important results columns:

In [4]:
model_col = "config_model_name"
task_col = "config_task_name"

feature_subset_col = "config_feature_subset"
predictions_path_col = "predictions_path"

Helper function to parse each dictionary containing benchmark results:

In [5]:
from utils import (
    num_features_helper,
    parse_model_name,
    get_non_instruction_tuned_name,
    prettify_model_name,
)

def parse_results_dict(dct) -> dict:
    """Parses results dict and brings all information to the top-level."""

    # Make a copy so we don't modify the input object
    dct = dct.copy()

    # Discard plots' paths
    dct.pop("plots", None)

    # Bring configs to top-level
    config = dct.pop("config", {})
    for key, val in config.items():
        dct[f"config_{key}"] = val

    # Parse model name
    dct[model_col] = parse_model_name(dct[model_col])
    dct["base_name"] = get_non_instruction_tuned_name(dct[model_col])
    dct["name"] = prettify_model_name(dct[model_col])

    # Is instruction-tuned model?
    dct["is_inst"] = dct["base_name"] != dct[model_col]

    # Log number of features
    dct["num_features"] = num_features_helper(dct[feature_subset_col], max_features_return=-1)
    dct["uses_all_features"] = (dct[feature_subset_col] is None) or (dct["num_features"] == -1)

    if dct[feature_subset_col] is None:
        dct[feature_subset_col] = "full"

    # Assert all results are at the top-level
    assert not any(isinstance(val, dict) for val in dct.values())
    return dct


Iteratively search the root directory for results files matching the given regex:

In [6]:
from utils import find_files, load_json

# Results file name pattern
pattern = r'^results.bench-(?P<hash>\d+)[.]json$'

# Find results files and aggregate
results = {}
for file_path in tqdm(find_files(RESULTS_ROOT_DIR, pattern)):
    results[Path(file_path).parent.name] = parse_results_dict(load_json(file_path))

if len(results) == 0:
    raise RuntimeError(f"Couldn't find any results at {RESULTS_ROOT_DIR}")
else:
    print(f"Found {len(results)} benchmark results.")

0it [00:00, ?it/s]

Found 100 benchmark results.


Aggregate results into a single DataFrame, and generate a unique identifier for each row:

In [7]:
df = pd.DataFrame(list(results.values()))

def row_id(row) -> str:
    """Unique row identifier."""
    return f"{row[model_col]}__{row[task_col]}__{row['num_features']}"

df["id"] = df.apply(row_id, axis=1)
df = df.set_index("id", drop=True, verify_integrity=True)

print(f"{df.shape=}")
df.sample(3)

df.shape=(100, 58)


,accuracy,accuracy_diff,accuracy_ratio,balanced_accuracy,balanced_accuracy_diff,balanced_accuracy_ratio,brier_score_loss,ece,ece_quantile,equalized_odds_diff,...,config_population_filter,config_reuse_few_shot_examples,config_seed,config_task_hash,config_task_name,base_name,name,is_inst,num_features,uses_all_features
id,,,,,,,,,,,,,,,,,,,,,
gemma-2-9b-it__ACSIncome__-1,0.780030,0.096459,0.888661,0.753813,0.249373,0.688385,0.203177,0.194715,0.179097,0.694508,...,None,False,42,2612382143,ACSIncome,gemma-2-9b,Gemma 2 9B (it),True,-1,True
gemma-1.1-7b-it__ACSPublicCoverage__-1,0.498748,0.185071,0.681191,0.474459,0.177263,0.696576,0.341414,0.296071,NaN,0.352822,...,None,False,42,1163226111,ACSPublicCoverage,gemma-7b,Gemma 7B (it),True,-1,True
gemma-7b__ACSPublicCoverage__-1,0.493723,0.079019,0.859522,0.504064,0.075661,0.865517,0.233823,0.148669,NaN,0.167430,...,None,False,42,1163226111,ACSPublicCoverage,gemma-7b,Gemma 7B,False,-1,True


Drop potential duplicates:

In [8]:
parsed_df = df.drop_duplicates(subset=["name", "is_inst", "num_features", task_col])
if len(parsed_df) != len(df):
    print(f"Found {len(df) - len(parsed_df)} duplicates! dropping rows...")
    df = parsed_df

Load scores DFs and analyze score distribution:

In [9]:
def load_model_scores_df(df_row: pd.Series) -> pd.DataFrame:
    """Loads csv containing model scores corresponding to the given DF row."""
    if predictions_path_col in df_row and not pd.isna(df_row[predictions_path_col]):
        return pd.read_csv(df_row[predictions_path_col], index_col=0)
    return None

In [10]:
from folktexts.evaluation import compute_best_threshold
from sklearn import metrics
from scipy import stats

# Number of samples used to fit the one-parameter binarization threshold!
N = 100

fit_thresh_col = f"fit_thresh_on_{N}"
fit_acc_col = f"fit_thresh_accuracy"

optimal_thres_col = "optimal_thresh"
optimal_acc_col = "optimal_thresh_accuracy"

score_stdev_col = "score_stdev"
score_mean_col = "score_mean"

scores_stats = dict()
for row_id, row in tqdm(df.iterrows(), total=len(df)):

    # Load model scores
    scores_df = load_model_scores_df(row)
    if scores_df is None:
        logging.error(f"Couldn't find scores for {row_id}")

    # Extract scores and labels
    risk_scores = scores_df["risk_score"].to_numpy()
    labels = scores_df["label"].to_numpy()

    # Sample N rows to fit threshold
    scores_df_sample = scores_df.sample(n=N, random_state=42)

    # Compute optimal threshold on each data sample
    fit_thr = compute_best_threshold(y_true=scores_df_sample["label"], y_pred_scores=scores_df_sample["risk_score"])
    opt_thr = compute_best_threshold(y_true=labels, y_pred_scores=risk_scores)

    # Evaluate accuracy
    fit_acc = metrics.accuracy_score(labels, (risk_scores >= fit_thr).astype(int))
    opt_acc = metrics.accuracy_score(labels, (risk_scores >= opt_thr).astype(int))

    # Save results
    scores_stats[row_id] = {
        fit_thresh_col: fit_thr,
        fit_acc_col: fit_acc,
        optimal_thres_col: opt_thr,
        optimal_acc_col: opt_acc,
        score_stdev_col: np.std(risk_scores),
        score_mean_col: np.mean(risk_scores),
    }

  0%|          | 0/100 [00:00<?, ?it/s]

Update results DF with scores statistics:

In [11]:
scores_stats_df = pd.DataFrame(scores_stats.values(), index=list(scores_stats.keys()))

results_df = pd.concat((df, scores_stats_df), axis="columns")
results_df.sample(2)

,accuracy,accuracy_diff,accuracy_ratio,balanced_accuracy,balanced_accuracy_diff,balanced_accuracy_ratio,brier_score_loss,ece,ece_quantile,equalized_odds_diff,...,name,is_inst,num_features,uses_all_features,fit_thresh_on_100,fit_thresh_accuracy,optimal_thresh,optimal_thresh_accuracy,score_stdev,score_mean
Meta-Llama-3-8B-Instruct__ACSMobility__-1,0.702966,0.226670,0.733909,0.503749,0.125906,0.774123,0.221827,0.152255,0.150708,0.402934,...,Llama 3 8B (it),True,-1,True,0.874048,0.534367,0.874078,0.53142,0.171341,0.812632
gemma-2-9b__ACSIncome__-1,0.402944,0.220766,0.514724,0.527615,0.065248,0.885922,0.246766,0.256795,0.254430,0.134841,...,Gemma 2 9B,False,-1,True,0.562223,0.722529,0.592505,0.74000,0.059786,0.575166


Check if any results are missing:

In [12]:
# print(results_df.groupby([model_col])[task_col].count())
for m in results_df[model_col].unique():
    for t in results_df[task_col].unique():
        match_ = results_df[(results_df[model_col] == m) & (results_df[task_col] == t)]
        if len(match_) != 1:
            print(f"Couldn't find results for m={m}, t={t}")

Finally, save results DF to the results root directory:

In [13]:
from utils import get_current_timestamp
results_df.to_csv(Path(RESULTS_ROOT_DIR) / f"aggregated_results.{get_current_timestamp()}.csv")

---